In [1]:
import pandas as pd

rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet')
rides.head(10)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
5,2022-01-01 00:40:15,138
6,2022-01-01 00:20:50,233
7,2022-01-01 00:13:04,238
8,2022-01-01 00:30:02,166
9,2022-01-01 00:48:52,236


In [2]:
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')

In [3]:
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [6]:
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()

In [8]:
agg_rides.rename(columns={0: 'rides'}, inplace=True)
agg_rides

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,4,11
1,2022-01-01 00:00:00,7,6
2,2022-01-01 00:00:00,10,1
3,2022-01-01 00:00:00,12,2
4,2022-01-01 00:00:00,13,12
...,...,...,...
66863,2022-01-31 23:00:00,261,4
66864,2022-01-31 23:00:00,262,8
66865,2022-01-31 23:00:00,263,26
66866,2022-01-31 23:00:00,264,24


In [9]:
from tqdm import tqdm

In [14]:
def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H'
    )
    output = pd.DataFrame()

    for location_id in tqdm(location_ids):

        #keep only rides for this location_id
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]

        #quickway to fill missing values with 0 in a series
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.inex)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)

        #add back location_id
        agg_rides_i['pickup_location_id'] = location_id
        
        output = pd.concat(output, agg_rides_i)

    #move the purchase_day from the index to dataframe colum
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})

    return output

        
    

add_missing_slots(agg_rides)

 26%|██▋       | 68/257 [00:00<00:00, 673.21it/s]

              pickup_hour  rides
0     2022-01-01 00:00:00     11
97    2022-01-01 01:00:00     15
206   2022-01-01 02:00:00     26
308   2022-01-01 03:00:00      8
405   2022-01-01 04:00:00      9
...                   ...    ...
66315 2022-01-31 18:00:00     16
66418 2022-01-31 19:00:00      7
66519 2022-01-31 20:00:00      1
66698 2022-01-31 22:00:00      1
66786 2022-01-31 23:00:00      3

[622 rows x 2 columns]
              pickup_hour  rides
1     2022-01-01 00:00:00      6
98    2022-01-01 01:00:00      4
207   2022-01-01 02:00:00      7
309   2022-01-01 03:00:00      7
406   2022-01-01 04:00:00      3
...                   ...    ...
66316 2022-01-31 18:00:00      3
66419 2022-01-31 19:00:00      5
66520 2022-01-31 20:00:00      2
66699 2022-01-31 22:00:00      1
66787 2022-01-31 23:00:00      3

[643 rows x 2 columns]
              pickup_hour  rides
2     2022-01-01 00:00:00      1
310   2022-01-01 03:00:00      1
407   2022-01-01 04:00:00      1
1112  2022-01-01 12:00:00   

100%|██████████| 257/257 [00:00<00:00, 693.63it/s]

              pickup_hour  rides
481   2022-01-01 04:00:00      1
2906  2022-01-02 09:00:00      1
3089  2022-01-02 11:00:00      1
3587  2022-01-02 16:00:00      1
4221  2022-01-02 23:00:00      1
...                   ...    ...
62293 2022-01-29 23:00:00      1
62385 2022-01-30 00:00:00      1
65423 2022-01-31 09:00:00      1
65958 2022-01-31 14:00:00      1
66178 2022-01-31 16:00:00      1

[76 rows x 2 columns]
              pickup_hour  rides
484   2022-01-01 04:00:00      1
9082  2022-01-05 07:00:00      1
16996 2022-01-08 23:00:00      2
18103 2022-01-09 12:00:00      1
18195 2022-01-09 13:00:00      1
19867 2022-01-10 09:00:00      1
20186 2022-01-10 12:00:00      1
20293 2022-01-10 13:00:00      2
22600 2022-01-11 15:00:00      1
26078 2022-01-13 08:00:00      1
27635 2022-01-14 00:00:00      1
28194 2022-01-14 07:00:00      1
28750 2022-01-14 12:00:00      1
29063 2022-01-14 15:00:00      1
35005 2022-01-17 11:00:00      1
37513 2022-01-18 15:00:00      1
37712 2022-01-18 17:

In [27]:
agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == 4, ['pickup_hour', 'rides']]
agg_rides_i

,pickup_hour,rides
0,2022-01-01 00:00:00,11
97,2022-01-01 01:00:00,15
206,2022-01-01 02:00:00,26
308,2022-01-01 03:00:00,8
405,2022-01-01 04:00:00,9
...,...,...
66315,2022-01-31 18:00:00,16
66418,2022-01-31 19:00:00,7
66519,2022-01-31 20:00:00,1
66698,2022-01-31 22:00:00,1


In [28]:
agg_rides_i.set_index('pickup_hour', inplace=True)

In [29]:
agg_rides_i

,rides
pickup_hour,
2022-01-01 00:00:00,11
2022-01-01 01:00:00,15
2022-01-01 02:00:00,26
2022-01-01 03:00:00,8
2022-01-01 04:00:00,9
...,...
2022-01-31 18:00:00,16
2022-01-31 19:00:00,7
2022-01-31 20:00:00,1


In [32]:
agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
agg_rides_i

,rides
pickup_hour,
2022-01-01 00:00:00,11
2022-01-01 01:00:00,15
2022-01-01 02:00:00,26
2022-01-01 03:00:00,8
2022-01-01 04:00:00,9
...,...
2022-01-31 18:00:00,16
2022-01-31 19:00:00,7
2022-01-31 20:00:00,1


In [33]:
full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H'
    )
agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)

In [34]:
agg_rides_i

,rides
2022-01-01 00:00:00,11
2022-01-01 01:00:00,15
2022-01-01 02:00:00,26
2022-01-01 03:00:00,8
2022-01-01 04:00:00,9
...,...
2022-01-31 19:00:00,7
2022-01-31 20:00:00,1
2022-01-31 21:00:00,0
2022-01-31 22:00:00,1
